In [1]:
using ParallelTemperingMonteCarlo, StaticArrays, DelimitedFiles

Broadly we need to save:

-Params which do not change

-Configs (the BC may change?)

-Results

In [ ]:
# function writeconfig(savefile , config{N,BC,T}, ) where {N,BC,T}
#     for row in config.pos
#         write(savefile, "$(row[1]) $(row[2]) $(ro2[3]) \n")
# end



Beginning with the static parameters which do not change:

In [ ]:

function writeparams(savefile, params, temp)# , ensemble, pot)
    write(savefile,"cycles: sample_rate: n_traj: n_atoms: t_i: t_f \n")
    write(savefile, "$(params.mc_cycles) $(params.mc_sample) $(params.n_traj) $(params.n_atoms) $(first(temp.t_grid)) $(last(temp.t_grid)) \n")
end
 

In [ ]:
function writeensemble(savefile,ensemble::NVT)
    write(savefile,"NVT: $(ensemble.n_atoms) $(ensemble.n_atom_moves) $(ensemble.n_atom_swaps) \n")
end
function writeensemble(savefile,ensemble::NPT)
    write(savefile,"NPT: $(ensemble.n_atoms) $(ensemble.n_atom_moves) $(ensemble.n_volume_moves) $(ensemble.n_atom_swaps) $(ensemble.pressure) \n")
end

In [ ]:
function writepotential(savefile,potential::Ptype) where Ptype <: AbstractDimerPotential
    write(savefile,"ELJ: $(potential.coeff) \n")
end
function writepotential(savefile,potential::Ptype) where Ptype <: AbstractDimerPotentialB
    write(savefile,"ELJB: $(potential.coeff_a) $(potential.coeff_b) $(potential.coeff_c) \n")
end
function writepotential(savefile,potential::Ptype) where Ptype <: EmbeddedAtomPotential
    write(savefile,"EAM: $(potential.n) $(potential.m) $(potential.ean) $(potential.eCam) \n")
end 
function writepotential(savefile,potential::Ptype) where Ptype <: AbstractMachineLearningPotential
    write("Define the potential elsewhere, it's too complicated for simple I/O \n")
end

In [ ]:
function 